Это задание поможет вам лучше разобраться в методах множественной проверки гипотез и позволит применить ваши знания на данных из реального биологического исследования.

В этом задании вы:
-вспомните, что такое t-критерий Стьюдента и для чего он применяется
-сможете применить технику множественной проверки гипотез и увидеть собственными глазами, как она работает на реальных данных
-почувствуете разницу в результатах применения различных методов поправки на множественную проверку

Основные библиотеки и используемые методы:
Библиотека scipy и основные статистические функции:http://docs.scipy.org/doc/scipy/reference/stats.html#statistical-functions

Библиотека statmodels для методов коррекции при множественном сравнении:

http://statsmodels.sourceforge.net/devel/stats.html

Статья, в которой рассматриваются примеры использования statsmodels для множественной проверки гипотез:

http://jpktd.blogspot.ru/2013/04/multiple-testing-p-value-corrections-in.html

Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)
В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.


В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.51.5.

Обратите внимание, что

применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0, когда они есть, и будут чаще отклонять H_0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [145]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint
from statsmodels.stats.weightstats import *

In [146]:
data = pd.read_csv('gene_high_throughput_sequencing.csv', sep = ',')

In [147]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [148]:
raw_normal = data[data.Diagnosis == 'normal']
raw_early = data[data.Diagnosis == 'early neoplasia']
raw_cancer = data[data.Diagnosis == 'cancer']

In [149]:
data_cancer = raw_cancer.drop(['Patient_id', 'Diagnosis'], axis=1)
data_normal = raw_normal.drop(['Patient_id', 'Diagnosis'],axis=1)
data_early = raw_early.drop(['Patient_id', 'Diagnosis'],axis=1)

In [150]:
def student_many(data1, data2):
    result = []
    for col in data1.columns:
        result.append(scipy.stats.ttest_ind(data1[col], data2[col], equal_var = False))
    return result

In [151]:
st1 = student_many(data_cancer, data_early)
st2 = student_many(data_normal, data_early)

In [152]:
test1_data = pd.DataFrame.from_records(st1)
test1_data.columns = ['stat','p']
test2_data = pd.DataFrame.from_records(st2)
test2_data.columns = ['stat','p']

In [153]:
test1_data['gens_name'] = np.mean(data_cancer, axis = 0).index
test1_data['early_mean'] = np.mean(data_early, axis = 0).values
test1_data['cancer_mean'] = np.mean(data_cancer, axis = 0).values


test2_data['gens_name'] = np.mean(data_normal, axis = 0).index
test2_data['normal_mean'] = np.mean(data_normal, axis = 0).values
test2_data['early_mean'] = np.mean(data_early, axis = 0).values

In [154]:
test1_data.head(10)

,stat,p,gens_name,early_mean,cancer_mean
0,-0.824849,0.413735,LOC643837,2.510894,2.186060
1,-0.452236,0.653429,LOC100130417,8.721781,8.190456
2,1.817266,0.079556,SAMD11,18.531325,23.692614
3,1.075985,0.287581,NOC2L,15.071854,16.468034
4,0.740329,0.463292,KLHL17,24.152469,25.035813
5,2.803258,0.007681,PLEKHN1,10.793160,13.870940
6,0.710101,0.481306,C1orf170,9.009031,9.677439
7,0.560597,0.578830,HES4,64.670644,67.162763
8,3.904478,0.000740,ISG15,37.296084,91.706112
9,-0.370542,0.712687,AGRN,75.707467,73.359613


In [155]:
from statsmodels.sandbox.stats.multicomp import multipletests 


alpha = 0.05/2
reject1, p_corrected1, a11, a21 = multipletests(test1_data.p, alpha = alpha, method = 'holm') 
reject2, p_corrected2, a12, a22 = multipletests(test2_data.p, alpha = alpha, method = 'holm') 

In [156]:
test1_data['rej_holm'] = reject1
test1_data['p_corr_holm'] = p_corrected1

test2_data['rej_holm'] = reject2
test2_data['p_corr_holm'] = p_corrected2

In [157]:
test2_data.head()

,stat,p,gens_name,normal_mean,early_mean,rej_holm,p_corr_holm
0,0.400289,0.690766,LOC643837,2.681277,2.510894,False,1.000000
1,-4.608766,0.000032,LOC100130417,4.368497,8.721781,False,0.500174
2,-1.929277,0.060273,SAMD11,15.159566,18.531325,False,1.000000
3,0.220542,0.826429,NOC2L,15.374351,15.071854,False,1.000000
4,-2.013201,0.049876,KLHL17,21.459886,24.152469,False,1.000000


In [158]:
def fct(c, t):
    if t > c:
        return float(t)/c
    return -float(c)/t

In [159]:
from math import fabs

test1_data['fct'] = map(fabs, map(fct, test1_data.early_mean.values, test1_data.cancer_mean.values))
test2_data['fct'] = map(fabs, map(fct, test2_data.normal_mean.values, test2_data.early_mean.values))

In [160]:
test1_data.head()

,stat,p,gens_name,early_mean,cancer_mean,rej_holm,p_corr_holm,fct
0,-0.824849,0.413735,LOC643837,2.510894,2.186060,False,1.0,1.148593
1,-0.452236,0.653429,LOC100130417,8.721781,8.190456,False,1.0,1.064871
2,1.817266,0.079556,SAMD11,18.531325,23.692614,False,1.0,1.278517
3,1.075985,0.287581,NOC2L,15.071854,16.468034,False,1.0,1.092635
4,0.740329,0.463292,KLHL17,24.152469,25.035813,False,1.0,1.036574


In [177]:
test1_ans = test1_data[(test1_data.rej_holm == True) & (test1_data.fct > 1.5)]
test2_ans = test2_data[(test2_data.rej_holm == True) & (test2_data.fct > 1.5)]

In [178]:
print test2_ans.shape[0], test1_ans.shape[0]

2 77


In [163]:
reject3, p_corrected3, a11, a21 = multipletests(test1_data.p, alpha = alpha, method = 'fdr_bh') 
reject4, p_corrected4, a12, a22 = multipletests(test2_data.p, alpha = alpha, method = 'fdr_bh') 

In [164]:
test1_data['rej_bh'] = reject3
test1_data['p_corr_bh'] = p_corrected3

test2_data['rej_bh'] = reject4
test2_data['p_corr_bh'] = p_corrected4

In [180]:
test3_ans = test1_data[(test1_data.rej_bh == True) & (test1_data.fct > 1.5)]
test4_ans = test2_data[(test2_data.rej_bh == True) & (test2_data.fct > 1.5)]

print test4_ans.shape[0], test3_ans.shape[0]

4 524


In [171]:
print test2_data[test2_data.p < 0.05].shape[0], test1_data[test1_data.p < 0.05].shape[0]

1575 3490


In [172]:
def write_answer(ans, name):
    with open(name, "w") as fout:
        fout.write(str(ans))

In [181]:
write_answer(test2_data[test2_data.p < 0.05].shape[0], 'bio1.txt')
write_answer(test1_data[test1_data.p < 0.05].shape[0], 'bio2.txt')
write_answer(test2_ans.shape[0], 'bio3.txt')
write_answer(test1_ans.shape[0], 'bio4.txt')
write_answer(test4_ans.shape[0], 'bio5.txt')
write_answer(test3_ans.shape[0], 'bio6.txt')